In [9]:
from IPython.core.magic import register_cell_magic
import os
import subprocess

@register_cell_magic
def cuda(line, cell):
    # 1. 把 Cell 里的代码保存为文件
    filename = "cuda_code.cu"
    with open(filename, "w") as f:
        f.write(cell)

    # 2. 编译命令 (自动加上了 -arch=sm_75 修复你的报错)
    # 这里的 -arch=sm_75 是专门针对 Colab T4 显卡的
    compile_cmd = "/usr/local/cuda/bin/nvcc -arch=sm_75 -o cuda_code cuda_code.cu"
    
    # 3. 执行编译
    print(f"🔨 Compiling with: {compile_cmd}")
    result = subprocess.run(compile_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode != 0:
        print("❌ Compilation Failed!")
        print(result.stderr)
        return

    # 4. 运行编译好的程序
    print("🚀 Running...")
    run_cmd = "./cuda_code"
    run_result = subprocess.run(run_cmd, shell=True, capture_output=True, text=True)
    
    # 5. 输出结果
    print(run_result.stdout)
    if run_result.stderr:
        print("Runtime Errors:", run_result.stderr)

print("✅ 自定义 %%cuda 已激活！现在你可以直接写代码了，无需任何参数。")

✅ 自定义 %%cuda 已激活！现在你可以直接写代码了，无需任何参数。


In [16]:
# 1. 安装必要的构建工具
!pip install ninja

import torch
from torch.utils.cpp_extension import load_inline
import os

# 2. 清理缓存 (防止旧的错误构建干扰)
!rm -rf /root/.cache/torch_extensions/

# 3. 再次确认 CUDA 可用
if not torch.cuda.is_available():
    print("❌ 错误: 未检测到 GPU，请在 Colab 菜单 'Runtime' -> 'Change runtime type' 中选择 GPU。")
else:
    print(f"✅ 检测到 GPU: {torch.cuda.get_device_name(0)}")

# 4. 定义代码
cuda_src = """
__global__ void square_matrix_kernel(const float* input, float* output, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = input[idx] * input[idx];
    }
}

void square_matrix(torch::Tensor input, torch::Tensor output) {
    int size = input.numel();
    const int threads = 256;
    const int blocks = (size + threads - 1) / threads;
    square_matrix_kernel<<<blocks, threads>>>(input.data_ptr<float>(), output.data_ptr<float>(), size);
}
"""

cpp_src = "void square_matrix(torch::Tensor input, torch::Tensor output);"

# 5. 编译 (关键修改：加入了 verbose=True)
try:
    square_module = load_inline(
        name='square_extension',
        cpp_sources=cpp_src,
        cuda_sources=cuda_src,
        functions=['square_matrix'],
        with_cuda=True,
        extra_cuda_cflags=["-O3"],
        verbose=True  # 🔥 关键：这会把底层的编译日志打印出来
    )
    print("✅ 编译成功！")

    # 测试
    x = torch.randn(1000).cuda()
    y = torch.zeros_like(x)
    square_module.square_matrix(x, y)
    print(f"验证结果: {torch.allclose(y, x*x)}")

except Exception as e:
    print("\n❌ 编译依然失败。请查看上方的详细日志 (Look at the log above)。")
    # 如果是因为环境里没有 gcc/g++，有时需要安装 (Colab通常自带，但以防万一)
    # print(e)

✅ 检测到 GPU: Tesla T4
✅ 编译成功！
验证结果: True


# Naive VS Tiled

In [26]:
import torch
from torch.utils.cpp_extension import load_inline
from torch.utils.cpp_extension import load
import os
import shutil
import sysconfig
from pathlib import Path

cache_dir = os.path.join(os.path.expanduser("~"), ".cache/torch_extensions")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
print("🧹已清理编译缓存")

# 获取 Python.h 所在的真实路径
# python_include = sysconfig.get_path('include')
# print(f"🔍 Python.h 应该在这里: {python_include}")

# 强制设置环境变量，让编译器一定能看到它
# os.environ['CPLUS_INCLUDE_PATH'] = python_include + os.pathsep + os.environ.get('CPLUS_INCLUDE_PATH', '')
# print("✅ 已强制注入头文件路径")

# --------------------------------------------------
# 1. CUDA 源码
# --------------------------------------------------
root = Path.cwd()
print(f"🔍 正在从 {root} 读取 CUDA 源码...")
cuda_source = ((root / "GEMM" / "NaiveGEMM.cu").read_text() + "\n"
                + (root / "GEMM" / "TiledGEMM.cu").read_text() + "\n" 
                + (root / "GEMM" / "RegisterTiledGEMM.cu").read_text() + "\n"
                + (root / "GEMM" / "HOST" /"NaiveGEMM.cu").read_text() + "\n"
                + (root / "GEMM" / "HOST" /"TiledGEMM.cu").read_text() + "\n"
                )


cpp_source = """
void run_sgemm_naive(torch::Tensor A, torch::Tensor B, torch::Tensor C);
void run_sgemm_tiled(torch::Tensor A, torch::Tensor B, torch::Tensor C);
"""

# ------------------------------------------------------------------
# 2. 编译模块
# ------------------------------------------------------------------
print(" 正在编译 CUDA 算子...")
gemm_module = load_inline(
    name='sgemm_comparison',
    cpp_sources=cpp_source,
    cuda_sources=cuda_source,
    functions=['run_sgemm_naive', 'run_sgemm_tiled'],
    with_cuda=True,
    extra_cuda_cflags=["-O3"]
)
print(" 编译完成！")

# ------------------------------------------------------------------
# 3. 性能测试 (Benchmark)
# ------------------------------------------------------------------
# 设置矩阵大小 (2048x2048 是一个比较能显现差距的大小)
M, N, K = 2048, 2048, 2048
A = torch.randn(M, K).cuda()
B = torch.randn(K, N).cuda()
C_naive = torch.zeros(M, N).cuda()
C_tiled = torch.zeros(M, N).cuda()

# 辅助函数：计时
def benchmark(func, name, *args):
    # 预热
    func(*args)
    torch.cuda.synchronize()
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    
    start.record()
    for _ in range(10): # 跑 10 次取平均
        func(*args)
    end.record()
    torch.cuda.synchronize()
    
    avg_time = start.elapsed_time(end) / 10.0
    print(f"[{name}] 平均耗时: {avg_time:.2f} ms")
    return avg_time

print(f"\n 开始测试 (矩阵大小: {M}x{N}x{K})...")

# 1. 跑 Naive
time_naive = benchmark(gemm_module.run_sgemm_naive, "Naive GEMM", A, B, C_naive)

# 2. 跑 Tiled
time_tiled = benchmark(gemm_module.run_sgemm_tiled, "Tiled GEMM", A, B, C_tiled)

# 3. 验证正确性
is_correct = torch.allclose(C_naive, C_tiled, atol=1e-2)
print(f"\n 结果匹配? {is_correct}")

# 4. 计算加速比
if is_correct:
    print(f"⚡ 加速比: {time_naive / time_tiled:.2f}x (Tiled 比 Naive 快了这么多)")
else:
    print(" 结果不正确，请检查代码逻辑！")

🧹已清理编译缓存
🔍 正在从 /content 读取 CUDA 源码...


FileNotFoundError: [Errno 2] No such file or directory: '/content/GEMM/NaiveGEMM.cu'

In [19]:
import os
import subprocess
import torch
from torch.utils.cpp_extension import include_paths, library_paths

# 获取 Python.h 所在的真实路径
python_include = sysconfig.get_path('include')
print(f"🔍 Python.h 应该在这里: {python_include}")

# 强制设置环境变量，让编译器一定能看到它
os.environ['CPLUS_INCLUDE_PATH'] = python_include + os.pathsep + os.environ.get('CPLUS_INCLUDE_PATH', '')
print("✅ 已强制注入头文件路径，请重试编译。")
# 1. 准备一段最简单的 CUDA 代码
code = """
#include <torch/extension.h>
__global__ void test_kernel(float* x) { *x = 1.0; }
void test_func(torch::Tensor x) { test_kernel<<<1,1>>>(x.data_ptr<float>()); }
PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) { m.def("test_func", &test_func); }
"""

# 2. 写入文件
with open("test_manual.cu", "w") as f:
    f.write(code)

# 3. 拼装编译命令 (模拟 PyTorch 的行为)
# 获取 PyTorch 的头文件路径
includes = [f"-I{p}" for p in include_paths()]
# 获取 PyTorch 的库路径
libs = [f"-L{p}" for p in library_paths()]

# 关键：手动调用 nvcc
cmd = [
    "nvcc", "-c", "test_manual.cu", "-o", "test_manual.o",
    "-x", "cu", "-Xcompiler", "-fPIC", "-std=c++17",
    "--expt-relaxed-constexpr"
] + includes

print("🔍 正在尝试手动编译 .o 文件...")
res = subprocess.run(cmd, capture_output=True, text=True)

if res.returncode != 0:
    print("❌ 手动编译失败！真正的错误如下：")
    print("="*40)
    print(res.stderr)
    print("="*40)
else:
    print("✅ 手动编译成功！nvcc 编译器是正常的。")
    print("问题出在 PyTorch 的 load_inline 封装上。")

🔍 Python.h 应该在这里: /usr/include/python3.12
✅ 已强制注入头文件路径，请重试编译。
🔍 正在尝试手动编译 .o 文件...
✅ 手动编译成功！nvcc 编译器是正常的。
问题出在 PyTorch 的 load_inline 封装上。
